# Purpose:

Update: LLM (Mistral quotes facts stated in gpt4 summaries which imo is higher quality data.

The purpose if this notebook is to get a list of prompts mapped with instructions and GPT4Summaries.

GPT4Summaries are extracted from persuade 2 corpus ([Check out this notebook on analysis behind this](https://www.kaggle.com/code/nbroad/persuade-train-essays-analysis/data)).

The intuition is that a student summarizes the given supplemental article before writing the essay according to the instructions provided.

Note: I know they don't use these to generate data in the competition...but I'm hoping this will give us better quality data.

In [1]:
from pandas import read_csv, concat

train_essays = read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv', chunksize=10000)
train_essays = concat(train_essays)

train_prompts = read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv', chunksize=10000)
train_prompts = concat(train_prompts)

test_essays = read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv', chunksize=10000)
test_essays = concat(test_essays)

In [2]:
train_essays.text.str.len().describe()

count    1378.000000
mean     3169.050798
std       920.588198
min      1356.000000
25%      2554.250000
50%      2985.500000
75%      3623.750000
max      8436.000000
Name: text, dtype: float64

In [3]:
train_essays.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [4]:
train_prompts.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [5]:
test_essays.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [6]:
print(train_prompts.iloc[0].instructions.replace('. ', '.\n'))

Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage.
Your essay must be based on ideas and information that can be found in the passage set.
Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response.
Be sure to use evidence from multiple sources; and avoid overly relying on one source.
Your response should be in the form of a multiparagraph essay.
Write your essay in the space provided.


In [7]:
persuade1 = read_csv('/kaggle/input/persaude-corpus-2/persuade_corpus_1.0.csv', chunksize=10000)
persuade1 = concat(persuade1)
persuade2 = read_csv('/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv', chunksize=10000)
persuade2 = concat(persuade2)

In [8]:
persuade2.head()

,essay_id_comp,full_text,holistic_essay_score,word_count,prompt_name,task,assignment,source_text,gender,grade_level,ell_status,race_ethnicity,economically_disadvantaged,student_disability_status
0,423A1CA112E2,Phones\n\nModern humans today are always on th...,3,378,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
1,BC75783F96E3,This essay will explain if drivers should or s...,4,432,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
2,74C8BC7417DE,Driving while the use of cellular devices\n\nT...,2,179,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,F,NaN,NaN,White,NaN,NaN
3,A8445CABFECE,Phones & Driving\n\nDrivers should not be able...,3,221,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,Black/African American,NaN,NaN
4,6B4F7A0165B9,Cell Phone Operation While Driving\n\nThe abil...,4,334,Phones and driving,Independent,Today the majority of humans own and operate c...,NaN,M,NaN,NaN,White,NaN,NaN


In [9]:
persuade2.essay_id_comp.size

25996

In [10]:
persuade2.word_count.describe()

count    25996.000000
mean       418.129635
std        275.110310
min        146.000000
25%        269.000000
50%        375.000000
75%        511.000000
max      14818.000000
Name: word_count, dtype: float64

In [11]:
persuade2.holistic_essay_score.describe()

count    25996.000000
mean         3.315010
std          1.160473
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          6.000000
Name: holistic_essay_score, dtype: float64

In [12]:
instructions = persuade2.drop_duplicates(subset=['essay_id_comp']).reset_index(drop=True)[['holistic_essay_score', 'prompt_name', 'assignment']]

In [13]:
instructions = instructions.drop_duplicates(subset=['prompt_name', 'assignment']).reset_index(drop=True)

In [14]:
sources = read_csv('/kaggle/input/persaude-corpus-2/sources.csv', chunksize=10000)
sources = concat(sources)

In [15]:
sources.head()

,prompt_name,title,author,url,gpt4_summary
0,Car-free cities,"In German Suburb, Life Goes On Without Cars",Elisabeth Rosenthal,https://www.nytimes.com/2009/05/12/science/ear...,"Vauban, an experimental district near Freiburg..."
1,Car-free cities,Paris bans driving due to smog,Robert Duffer,https://scoringguides.cambiumast.com/pdfs/Grad...,"Paris faced severe pollution, prompting a part..."
2,Car-free cities,Car-free day is spinning into a big hit in Bogota,Andrew Selsky,https://scoringguides.cambiumast.com/pdfs/Grad...,"In Bogotá, Colombia, a successful ""Day Without..."
3,Car-free cities,The end of car culture,Elisabeth Rosenthal,https://www.nytimes.com/2013/06/30/sunday-revi...,President Obama's goals to reduce greenhouse g...
4,Does the electoral college work?,What is the Electoral College?,The Office of the Federal Register,https://www.archives.gov/electoral-college/about,"The Electoral College, established in the U.S...."


In [16]:
sources = sources[['prompt_name', 'gpt4_summary']]

In [17]:
sources.head()

,prompt_name,gpt4_summary
0,Car-free cities,"Vauban, an experimental district near Freiburg..."
1,Car-free cities,"Paris faced severe pollution, prompting a part..."
2,Car-free cities,"In Bogotá, Colombia, a successful ""Day Without..."
3,Car-free cities,President Obama's goals to reduce greenhouse g...
4,Does the electoral college work?,"The Electoral College, established in the U.S...."


In [18]:
sources.prompt_name.unique()

array(['Car-free cities', 'Does the electoral college work?',
       'Exploring Venus', 'The Face on Mars',
       'Facial action coding system', '"A Cowboy Who Rode the Waves"',
       'Driverless cars'], dtype=object)

In [19]:
instructions = instructions.query('prompt_name in @sources.prompt_name.unique()').drop(columns=['holistic_essay_score']).reset_index(drop=True)

In [20]:
prompts = sources.merge(instructions, on='prompt_name', how='outer')

In [21]:
prompts = prompts.fillna( '')

In [22]:
prompts

,prompt_name,gpt4_summary,assignment
0,Car-free cities,"Vauban, an experimental district near Freiburg...",Write an explanatory essay to inform fellow ci...
1,Car-free cities,"Paris faced severe pollution, prompting a part...",Write an explanatory essay to inform fellow ci...
2,Car-free cities,"In Bogotá, Colombia, a successful ""Day Without...",Write an explanatory essay to inform fellow ci...
3,Car-free cities,President Obama's goals to reduce greenhouse g...,Write an explanatory essay to inform fellow ci...
4,Does the electoral college work?,"The Electoral College, established in the U.S....",Write a letter to your state senator in which ...
5,Does the electoral college work?,"The electoral college, a system where voters s...",Write a letter to your state senator in which ...
6,Does the electoral college work?,"The Electoral College, often criticized as an ...",Write a letter to your state senator in which ...
7,Exploring Venus,,"In ""The Challenge of Exploring Venus,"" the aut..."
8,The Face on Mars,"Twenty-five years ago, NASA's Viking 1 spacecr...",You have read the article 'Unmasking the Face ...
9,Facial action coding system,,"In the article ""Making Mona Lisa Smile,"" the a..."


In [23]:
from itertools import repeat

prompts = concat([*repeat(prompts, 10)])

In [24]:
prompts.iloc[0]

prompt_name                                       Car-free cities
gpt4_summary    Vauban, an experimental district near Freiburg...
assignment      Write an explanatory essay to inform fellow ci...
Name: 0, dtype: object

In [25]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.4/613.4 kB 33.3 MB/s eta 0:00:00

[notice] A new r

[Reference on using Mistral](https://www.kaggle.com/code/phanisrikanth/generate-synthetic-essays-with-mistral-7b-instruct)

In [26]:
test = prompts.iloc[0]

In [27]:
test.assignment

'Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.'

In [28]:
test.gpt4_summary

"Vauban, an experimental district near Freiburg, Germany, is pioneering suburban living without reliance on cars. Established in 2006, it prohibits street parking, driveways, and home garages, allowing car ownership but limiting parking to large garages on the development's edge, costing $40,000 per space. This innovative concept has led 70% of Vauban’s families to live without cars, with 57% selling their cars to move there. Emphasizing “smart planning,” Vauban is a model in reducing greenhouse gas emissions, a critical factor in combating global warming. The district, home to 5,500 residents, showcases how suburban areas can be more compact, accessible to public transportation, and less car-dependent. This approach aligns with efforts in the U.S. and Europe to develop suburbs that favor public transport and walking, with amenities within walking distance. However, despite these efforts, challenges remain in changing car-centric attitudes and adapting urban planning regulations to sup

In [29]:
!pip install jax==0.4.19

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: jax
    Found existing installation: jax 0.4.17
    Uninstalling jax-0.4.17:
      Successfully uninstalled jax-0.4.17

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [34]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [129]:
from langchain.prompts import PromptTemplate
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import CTransformers

import torch
import torch_xla.core.xla_model as xm
from multiprocessing import Pool
from pandas import DataFrame, Series


def generate_essays(prompts):
    model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
    device = xm.xla_device()
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    pipe_kwargs = lambda : {
        'max_new_tokens': 350, 
        'repetition_penalty': 1.1,
        "context_length" : 768,
        "temperature" : 0.9,
        "batch_size": 8,
    }
    #pipe = pipeline("text-generation", **pipe_kwargs())
    #pipe.tokenizer.pad_token_id = pipe.model.config.eos_token_id
    
    hf = CTransformers(model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF", 
                       config=pipe_kwargs())
    def generate_prompt(x : Series) -> str:
        return {
        "question": x.assignment,  
        "system" : "You are a student", 
        "instruction" : "Word limit minimum 512 words and maximum 3000 words. Refer to the summary provided for supplement context.",
        "info" : x.gpt4_summary
    }
    template = """<s>
    [INST] Your role
    {system} 
    [/INST]
    [INST] Your task: 
    {question} 
    [/INST]
    [INST] Instructions
    {instruction} 
    [/INST]
    [INST] Reference Summary:
    {info} 
    [/INST]
    </s>
    """
    prompt = PromptTemplate.from_template(template)
    chain = prompts.apply(lambda x: generate_prompt(x), axis=1)
    llm = prompt | hf
    return Series(llm.batch(chain.values.tolist()))

In [130]:
generated_dataset = generate_essays(prompts)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 14169.95it/s]


In [131]:
#~5 min -> 8 essays 
print(generated_dataset.iloc[-1])


    The article "The Challenge of Exploring Venus" discusses the challenges and potential benefits of studying Venus, a planet in our solar system that is similar to Earth in many ways. The author argues that despite the dangers of exploring Venus, it is still a worthy pursuit because it could lead to scientific breakthroughs and an understanding of our own planet's history and future.
    The article provides several examples of why studying Venus is important. Firstly, the author notes that Venus has a similar atmosphere to Earth, with clouds and temperatures that are similar to those on Earth. This makes it a potential model for understanding how Earth's climate has evolved over time and could also help us predict how our own planet might change in the future. Secondly, the author argues that studying Venus could lead to breakthroughs in fields such as space exploration and astronomy. For example, studying Venus could help us understand more about the formation of planets and the c

In [132]:
generated_dataset.to_csv('/kaggle/working/generated_dataset.csv')